In [ ]:
# Set TensorFlow version to 1.x
%tensorflow_version 1.x

# Print assigned GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install streamlit
!npm install localtunnel


# IMPORTANT: Ensure that the repository directory follows the path:
# "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"

In [ ]:
%cd "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"
!streamlit run human_pose_app.py &>/dev/null&
!npx localtunnel --port 8501 --subdomain 'coco-human-pose'

In [ ]:
# Download dataset to pre-specified folder on local VM instance
# Change dir to project dir
%cd "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"

import time
from datetime import datetime, timedelta

start = time.time()

!bash "./scripts/coco_dl.sh" /content/datasets

download_time = time.time() - start
print("Total download time: {}".format(str(timedelta(seconds=download_time))))

In [ ]:
!ls /content/datasets/coco/train2017 | wc -l
!ls /content/datasets/coco/val2017 | wc -l

In [ ]:
# Ensure that filesystem is set up correctly and will not be the bottleneck
import time
from datetime import datetime, timedelta

start = time.time()

import os
def print_files_in_dir(path):
    print(len(os.listdir(path)), " files at ", path)

print_files_in_dir('/content/datasets/coco/train2017/')
print_files_in_dir('/content/datasets/coco/val2017/')


setup_time = time.time() - start
print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))

if setup_time > 1:
    print("There appears to be a bottleneck with filesystem loading time. This may severely impact training speed.")

In [ ]:
# Toggle COLAB_TRAINING variable in constants file
# https://askubuntu.com/questions/20414/find-and-replace-text-within-a-file-using-commands
!sed -i.bak 's/COLAB_TRAINING = False/COLAB_TRAINING = True/g' constants.py 

# Remove backup file
!rm constants.py.bak

# Upgrade required imgaug package because of a bug on 0.2.9
!pip install --upgrade imgaug==0.4.0

In [ ]:
!python3 train.py --epochs 100 --batch 12 --hourglass 4 --subset 1.0 --loss "weighted_mse" --augment "none" --notes "sigma4"

# To resume, add the following to the above command, and point it to the subdirectory of the model checkpoints. Ensure everything matches
# --resume True --resume-epoch 15 --resume-subdir "2021-03-13-16h-38m_batchsize_12_hg_4"

# Note about subset: if you decide to run on a subset <1.0, there is no functionality to retrieve the same subset of data if the model is stopped then resumed. For this reason, subset should only be used for quick tests rather than reporting robust/repeatable results. 

# Note about loss function and augmentation: Ensure that they match across training sessions